## Model making

In [8]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)
# 2. 모델 및 토크나이저 로드 (8-bit 양자화)
model_name = "CarrotAI/Llama-3.2-Rabbit-Ko-1B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # QLoRA는 4bit 양자화를 사용
    # load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # 계산 precision (float16 또는 bfloat16 사용 가능)
    bnb_4bit_use_double_quant=True,       # 이중 양자화 활성화
    bnb_4bit_quant_type="nf4"             # NF4 양자화 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # BitsAndBytesConfig 추가
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  


# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

# model = get_peft_model(model, lora_config)


## Data Processing

In [72]:
def make_dataset(dataset, prompts, system_prompt):
    records = []
    for _, row in dataset.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue' : row.get('klue', None),
            'question_type' : row.get('question_type', None)
        }
        records.append(record)
            
    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    processed_dataset = []

    for i in range(len(dataset)):
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )
        # chat message 형식으로 변환
        processed_dataset.append(
            {
                "id": dataset[i]["id"],
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                    {"role": "assistant", "content": f"{dataset[i]['klue']}"}
                ],
                "label": dataset[i]['answer'],
                'type' : dataset[i]['question_type']
            }
        )
    processed_dataset = Dataset.from_pandas(pd.DataFrame(processed_dataset))

    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example["messages"])):
            output_texts.append(
                tokenizer.apply_chat_template(
                    example["messages"][i],
                    tokenize=False,
                )
            )
        return output_texts

    def tokenize(element):
        outputs = tokenizer(
            formatting_prompts_func(element),
            truncation=False,
            padding=False,
            return_overflowing_tokens=False,
            return_length=False,
        )
        return {
            "input_ids": outputs["input_ids"],
            "attention_mask": outputs["attention_mask"],
        }

    # 데이터 토큰화
    tokenized_dataset = processed_dataset.map(
        tokenize,

        batched=True,
        num_proc=1,
        load_from_cache_file=True,
        desc="Tokenizing",
    )

    # 데이터 분리
    tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["input_ids"]) <= 2048)  
    tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.10, seed=42)

    train_dataset = tokenized_dataset['train']
    eval_dataset = tokenized_dataset['test']


    train_dataset_token_lengths = [len(train_dataset[i]["input_ids"]) for i in range(len(train_dataset))]
    print(f"max token length: {max(train_dataset_token_lengths)}")
    print(f"min token length: {min(train_dataset_token_lengths)}")
    print(f"avg token length: {np.mean(train_dataset_token_lengths)}")

    # 데이터 확인
    return train_dataset, eval_dataset

In [73]:
from prompts import user_prompts
system_prompt = """지시에 따라 주어진 문제의 정답을 구하세요."""
prompts = user_prompts
dataset = pd.read_csv('datas/train+klue.csv')
train_dataset, eval_dataset = make_dataset(dataset,prompts, system_prompt)
# 여기서 프롬프트만 바꿔서 데이터셋을 만들기

Filter: 100%|██████████| 2031/2031 [00:01<00:00, 1441.06 examples/s]


max token length: 1664
min token length: 214
avg token length: 750.4258347016968


In [76]:
print(tokenizer.decode(train_dataset['input_ids'][5]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

지시에 따라 주어진 문제의 정답을 구하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

문제 유형:
기타

지문:
본문 1:
“전하, 게다가 우리들의 왕국에는 신께 도움이 되지 않는 불편함이 존재합니다. 바로 우리 백성들 중 많은 이들이 그대의 백성들이 가져오는 왕국의 상품과 물건을 간절히 원하고 있다는 것, 그런데 그대의 백성들은 자신들의 탐욕스러운 욕망을 만족시키기 위해 자유민이자 해방된 나의 백성들을 잡아가고 있다는 것입니다. 심지어 귀족과 왕의 친척까지도 잡아가 우리들의 왕국에 있는 백인들에게 팔고 있다는 것입니다.”
콩고의 아폰소 1세 국왕이 포르투갈의 주앙 3세 국왕에게 보낸 편지, 1526
출처 2:
“이번 원정에 많은 비용이 들었기에 빈 손으로 돌아간다면 합리적이지 못한 일이 될 것이다. 우리의 [주된] 바람은 신을 섬기는 것과 콩고 국왕을 기쁘게 하는 것이지만, 그럼에도 불구하고 콩고 국왕으로 하여금 노예가 됐건 구리가 됐건 상아가 됐건 배를 채워야 한다는 사실을 우리들의 이름으로 이해시켜야만 한다.”
포르투갈 마누엘 국왕의 콩고에 있는 사절에게 보낸 편지, 1512

질문:
편지에 설명된 상호 작용은 다음 중 어떤 맥락에서 가장 잘 이해되는가?

선택지:
1 - 포르투갈의 서아프리카 해안 탐험
2 - 사하라 이남 아프리카에서의 가톨릭 선교 활동
3 - 사하라 이남 아프리카의 국가 형성
4 - 사하라 이남 아프리카의 노예 무역 발전

출력 형식:
근거: 답변을 도출한 텍스트 # 정답 번호

문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때, 반드시 지문을 참고하세요.
반드시 지문에서 정답의 근거를 찾으세요.
반드시 출력 형식을 지키세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

근거 : 내용 형

## Train phase

In [51]:
import numpy as np
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def preprocess_logits_for_metrics(logits, labels):
    logits = logits if not isinstance(logits, tuple) else logits[0]
    logit_idx = [tokenizer.vocab["1"],
                    tokenizer.vocab["2"],
                    tokenizer.vocab["3"],
                    tokenizer.vocab["4"], 
                    tokenizer.vocab["5"]]
    logits = logits[:, -2, logit_idx] # -2: answer token, -1: eos token
    return logits


    # metric 계산 함수
def compute_metrics(evaluation_result):
    logits, labels = evaluation_result
    int_output_map = {"1": 0, "2": 1, "3": 2, "4": 3, "5": 4}


    # 토큰화된 레이블 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    print(labels)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    labels = [label.split('#')[-1].strip() for label in labels]
    labels = [int_output_map.get(label, -1) for label in labels] 

    # 소프트맥스 함수를 사용하여 로그트 변환
    probs = torch.nn.functional.softmax(torch.tensor(logits, dtype=torch.float32), dim=-1)

    predictions = np.argmax(probs, axis=-1)

    # 정확도 계산
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}


In [13]:
response_template = "assistant<|end_header_id|>"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

sft_config = SFTConfig(
    do_train=True,
    do_eval=True,
    lr_scheduler_type="cosine",
    max_seq_length = 2048,
    output_dir=f"./outputs + {model_name.split('/')[-1]}",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=2,
    save_only_model=True,
    report_to="none",
    fp16 = True,
    gradient_checkpointing = False, # 8B모델 돌릴때만 True로, 아니면 False로
    load_best_model_at_end = True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    args=sft_config,
    packing=False,
    peft_config = lora_config
)
torch.cuda.empty_cache()


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:463: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(


In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,1.430800,1.224366,0.421569,0.136430


TrainOutput(global_step=228, training_loss=1.3938894690128796, metrics={'train_runtime': 478.0686, 'train_samples_per_second': 3.822, 'train_steps_per_second': 0.477, 'total_flos': 8007966058217472.0, 'train_loss': 1.3938894690128796, 'epoch': 0.9983579638752053})

## Inference Phase

In [105]:
from ast import literal_eval
from datasets import Dataset

def make_test_dataset(test_df, prompt, system_prompt):
    # Flatten the JSON dataset
    records = []
    for _, row in test_df.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue': row.get('klue', None),
            'question_type': row.get('question_type', None)
        }
        records.append(record)

    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    
    # test_dataset를 빈 리스트로 초기화
    test_dataset = []

    # dataset의 각 항목에 대해 처리
    for i in range(len(dataset)):
        # choices_string 생성
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        
        # question_type에 맞는 prompt 선택
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형

        # user_message 생성
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )

        # len_choices는 choices의 길이로 계산
        len_choices = len(dataset[i]["choices"])

        # test_dataset에 항목 추가
        test_dataset.append(
            {
                "id": dataset[i]["id"],  # dataset[i]에서 'id' 가져오기
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                ],
                "label": dataset[i].get('answer', None),  # 'answer'가 없으면 None
                'type': dataset[i].get('question_type', None),
                "len_choices": len_choices,
            }
        )

    return test_dataset


## 1. Test dataset inference


In [106]:
from torch.utils.data import DataLoader

test_df = pd.read_csv('datas/test_processing_4.csv')
test_dataset = make_test_dataset(test_df, prompt, system_prompt)


In [113]:
test_dataset[700]

{'id': 'generation-for-nlp-785',
 'messages': [{'role': 'system', 'content': '지시에 따라 주어진 문제의 정답을 구하세요.'},
  {'role': 'user',
   'content': '\n문제 유형:\n이해형\n\n지문:\nI. MR = P = 완전 경쟁의 수평 함수에 대한 수요. II. 불완전 경쟁의 우하향 함수로서 P > MR. III. 수요와 가격이 불완전 경쟁의 수직 함수로 표시됨.\n\n질문:\n다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 불완전 경쟁 하에서 가격(P)이 한계 수익보다 큰 이유를 올바르게 설명하는 것은 무엇입니까?\n\n선택지:\n1 - I, II 및 III\n2 - I 및 III\n3 - I만 해당.\n4 - I 및 III\n\n출력 형식:\n근거: 답변을 도출한 텍스트 # 정답 번호\n\n문제는 무조건 1개의 정답만 있습니다.\n반드시 지문을 이해하고 이해한 내용을 바탕으로 정답을 고르세요.\n반드시 출력 형식을 지키세요.\n'}],
 'label': '',
 'type': '이해형',
 'len_choices': 4}

In [28]:
tokenizer.padding_side = 'left'


# 배치 데이터 로더를 위한 collate_fn
def collate_fn(batch):
    ids = [item["id"] for item in batch]
    messages = [item["messages"] for item in batch]
    labels = [item.get("label", None) for item in batch]  # 라벨이 존재할 경우만 가져옴
    return ids, messages, labels

# 데이터 로더 설정
batch_size = 8  # 배치 크기 설정
dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

generated_infer_results = []

with torch.inference_mode():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,  # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

100%|██████████| 1/1 [00:12<00:00, 12.68s/it]


In [30]:
generated_infer_results

,id,answer,label
0,generation-for-nlp-0,ette.,
1,generation-for-nlp-1,"속) 내용에 따라 내용이 다르며, 그 내용은 독서의 즐거움을 더 강화한다. 독서의 ...",
2,generation-for-nlp-2,�로(가) 중국에서 비롯된 유서(類書)는 고금의 서적에서 자료를 수집하고 항목별로 ...,
3,generation-for-nlp-3,�지,
4,generation-for-nlp-4,"�용은 지식의 역학적 진화를 수용하는 내용을 강조하고, ㉡에 대한 이해는 ㉡의 성격...",
5,generation-for-nlp-5,"�문\n\n㉡의 말에 따르면, '이수광'의 말은 '중국 유서'가 '중국 유서'의 지...",
6,generation-for-nlp-6,질문에 대한 답변은 다음과 같습니다.\n\n[근거: 답변] # 3 - 당대 지식을 ...,
7,generation-for-nlp-7,�상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상상...,


In [44]:
print(generated_infer_results.loc[0,'answer'])

ette.


In [22]:
generated_infer_results['answer'][6]

'근거: 답변을 도출한 내용에 따르면, (가)의 실학자들의 유서와 (나)의 유서는 모두 주자학의 관념적 사유를 반영한다. (나)의 유서는 주자학의 지적 영역 내에서 서학의 지식을 수용하는 내용을 보여주고, 이는 서학의 진보성의 토대가 중국이라는 서학 중국 원류설을 반영한 것이었다. 이는 (가)와 (나) 모두 서학의 지식의 역사적 의미를 강조한다. # [정답 번호]'

In [ ]:
import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = generated_infer_results
# 데이터프레임에 적용
result['text'] = result['answer']
result['answer'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['answer'] = result['answer'].apply(extract_last_digit)
result['answer'] = result['answer'].fillna(1)
result['answer'] = result['answer'].apply(lambda x: int(x))
submission = result[['id', 'text', 'answer']]
submission['answer'].value_counts()

### 2. Eval dataset으로 inference
#### 프롬프트를 바꿔가며 몇개 맞추나 보기

In [164]:
tokenizer.padding_side = 'left'
batch_size = 8  # 배치 크기 설정
dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

generated_infer_results = []

with torch.inference_mode():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,  # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )
        

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

100%|██████████| 26/26 [03:17<00:00,  7.59s/it]


In [169]:
from copy import deepcopy

import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = deepcopy(generated_infer_results)
# 데이터프레임에 적용
result['text'] = result['answer']
result['predict'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['predict'] = result['predict'].apply(extract_last_digit)
result['predict'] = result['predict'].fillna(1)
result['predict'] = result['predict'].apply(lambda x: int(x))

result = result[['text', 'predict', 'label',]]
result['question_type'] = eval_dataset['type']


In [170]:
result

,text,predict,label,question_type
0,"근거 : 내용에 따르면, 로열 베이비 조지 알렉산더 루이스 왕자가 태어난 후 아덴아...",1,1,기타
1,"근거 : 내용에 따르면, 문화스포츠계에서 가수 정기고와 박정현의 신곡 발매 계획이 ...",5,1,이해형
2,"근거 : 위로 기울어지는 총 공급 곡선은 공급의 감소를 나타내며, 이는 균형 물가 ...",1,1,기타
3,"근거 : Andy의 행동은 자기중심주의를 나타내며, 자신의 기대와 목표를 우선시하고...",1,4,기타
4,"근거 : 내용에 따르면, 코로나19 감염예방 홍보 가이드에는 마스크 착용 권장 안내...",1,1,이해형
...,...,...,...,...
199,"근거 : 카프의 어머니는 학교를 중단하는 것을 권장하며, 그의 열정을 살리기 위한 ...",1,1,이해형
200,"근거 : 내용에 따르면, 올랭프 드 구주는 혁명 시대에 여성은 일반적으로 지도자 역...",3,1,이해형
201,"근거 : 내용에 따르면, 11월 수출액은 작년 같은 달보다 0.2% 증가했다고 명확...",1,1,기타
202,"근거 : 내용 형식으로 보면, 이 지문은 아리스토텔레스와 플라톤의 철학적 우열 논쟁...",1,1,이해형
